In [55]:
import pandas as pd
import os

In [56]:
def clear_columns(data, columns):
    for c in columns:
        mask1 = data[c] == '\\N'
        mask2 = data[c].isnull()
        data = data.drop(index=(data[mask1 | mask2]).index)
    return data

In [57]:
title_ratings = pd.read_table("db_files/title_ratings.tsv")
title_ratings = title_ratings[title_ratings['numVotes'] >= 500]
title_ratings.shape

(125160, 3)

In [58]:
title_basic = pd.read_table("db_files/title_basic.tsv")
title_basic = title_basic[title_basic['titleType'] == 'movie']

C:\Users\ignacio\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [59]:
df = title_basic.join(title_ratings.set_index('tconst'), on='tconst').dropna()
df.shape

(57044, 11)

In [60]:
df = df.drop(columns='titleType')
df = clear_columns(df, df.columns.drop(['endYear','runtimeMinutes']))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57032 entries, 570 to 9719662
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          57032 non-null  object 
 1   primaryTitle    57032 non-null  object 
 2   originalTitle   57032 non-null  object 
 3   isAdult         57032 non-null  object 
 4   startYear       57032 non-null  object 
 5   endYear         57032 non-null  object 
 6   runtimeMinutes  57032 non-null  object 
 7   genres          57032 non-null  object 
 8   averageRating   57032 non-null  float64
 9   numVotes        57032 non-null  float64
dtypes: float64(2), object(8)
memory usage: 6.8+ MB


In [61]:
df['startYear'] = df['startYear'].apply(int)
df = df[df.startYear >= 1930]

df['endYear'] = df.apply(
    lambda row:row['startYear'] if row['endYear'] == '\\N' else int(row['endYear']), axis=1
)

df['runtimeMinutes'] = df.apply(
    lambda row: 0 if row['runtimeMinutes'] == '\\N' else row['runtimeMinutes'], axis=1
)
df.shape

(56577, 10)

In [62]:
df.head()
os.makedirs('db_files', exist_ok=True)
df.to_csv('db_files/movies_db_cleaned.csv')